<a href="https://colab.research.google.com/github/mogoweb/tensorflow-open_nsfw/blob/master/tensorRT_open_nsfw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

挂载谷歌云端硬盘：

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


切换当前目录为Google云端硬盘的项目文件夹：

In [0]:
import os

project_path = '/content/gdrive/My Drive/nvidia'  #change dir to your project folder
os.chdir(project_path)  #change dir

确定CUDA，Ubuntu，TensorFlow的版本：

In [3]:
!nvcc --version
!cat /etc/issue
import tensorflow as tf
print(tf.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
Ubuntu 18.04.2 LTS \n \l

1.13.1


下载TensorRT，由于nvidia官网下载需要注册、登录等诸多步骤，而且Google云端硬盘的上传速度比较慢，所以我放了一份拷贝到dropbox中，从dropbox下载要快得多：

In [4]:
!wget -O nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb https://www.dropbox.com/s/45pz13r4e8ip4bl/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb?dl=0

--2019-05-20 02:44:12--  https://www.dropbox.com/s/45pz13r4e8ip4bl/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/45pz13r4e8ip4bl/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb [following]
--2019-05-20 02:44:13--  https://www.dropbox.com/s/raw/45pz13r4e8ip4bl/nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucde3d211acf7d646f6d6c13f361.dl.dropboxusercontent.com/cd/0/inline/AhMamzIh2weX4PjyB9faTn3GPs0sEvzIn5n9XWvMt011tH--uYO43Qi5yznxbA4zFkd_dU75Wx715t-vRFppqKkQL25Is9m69byCmEZXKJxI2g/file# [following]
--2019-05-20 02:44:13--  https://ucde3d211acf7d

In [4]:
!dpkg -i nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb
!apt-key add /var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227/7fa2af80.pub
!apt-get update
!apt-get install -y --no-install-recommends libnvinfer5=5.1.2-1+cuda10.0
!apt-get install -y --no-install-recommends libnvinfer-dev=5.1.2-1+cuda10.0
!apt-get install tensorrt
!apt-get install python3-libnvinfer-dev
!apt-get install uff-converter-tf

Selecting previously unselected package nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227.
(Reading database ... 130824 files and directories currently installed.)
Preparing to unpack nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227 (1-1) ...
Setting up nv-tensorrt-repo-ubuntu1804-cuda10.0-trt5.1.2.2-rc-20190227 (1-1) ...
OK
Get:1 file:/var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227  InRelease
Ign:1 file:/var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227  InRelease
Get:2 file:/var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227  Release [574 B]
Get:2 file:/var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227  Release [574 B]
Get:3 file:/var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227  Release.gpg [819 B]
Get:3 file:/var/nv-tensorrt-repo-cuda10.0-trt5.1.2.2-rc-20190227  Release.gpg [819 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease 

克隆open_nsfw的tensorflow版本代码库：

In [8]:
!git clone https://github.com/mogoweb/tensorflow-open_nsfw.git

Cloning into 'tensorflow-open_nsfw'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 130 (delta 26), reused 46 (delta 22), pack-reused 79
Receiving objects: 100% (130/130), 21.13 MiB | 19.43 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [0]:
os.chdir(os.path.join(project_path, "tensorflow-open_nsfw"))

判断单个图片是否色情图片，请根据实际情况替换后面的图片路径：

In [6]:
!python classify_nsfw.py -m data/open_nsfw-weights.npy ../test/porn/23241c38-a7f7-44c0-9faa-4bed8dfff3a4.jpg

2019-05-20 08:36:45.008917: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-20 08:36:45.009204: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x32c71e0 executing computations on platform Host. Devices:
2019-05-20 08:36:45.009241: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-20 08:36:45.242438: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-20 08:36:45.242999: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x32c6dc0 executing computations on platform CUDA. Devices:
2019-05-20 08:36:45.243028: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-20 08:36:45.243365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

导出tensorflow模型图：

In [13]:
!python tools/export_graph.py -m ./data/open_nsfw-weights.npy -i tensor -f ./data/models/

2019-05-20 08:58:43.129808: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-20 08:58:43.130011: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x27c6520 executing computations on platform Host. Devices:
2019-05-20 08:58:43.130039: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-20 08:58:43.305385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-20 08:58:43.305906: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x27c5e40 executing computations on platform CUDA. Devices:
2019-05-20 08:58:43.305937: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-20 08:58:43.306294: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found de

判断测试集图片，图片位于指定目录下：

In [32]:
!python opt/benchmark_classify_nsfw.py -i ../test/porn -m ./data/models/frozen_open_nsfw.pb

Instructions for updating:
Use tf.gfile.GFile.
2019-05-20 09:38:23.213304: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-05-20 09:38:23.213517: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2812260 executing computations on platform Host. Devices:
2019-05-20 09:38:23.213568: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-20 09:38:23.370835: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-20 09:38:23.371418: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2811e40 executing computations on platform CUDA. Devices:
2019-05-20 09:38:23.371451: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-20 09:38:23.371799: I tensorflow/core/